***
# Projet n°7 - Implémentez un modèle de scoring
***
Notebook Jupyter réalisé par Dominique LEVRAY en Juillet/Août 2024

**3ème Notebook - Test_API**

Ce notebook intègre :
- Les tests de l'API de prédiction du modèle de scoring
***


In [10]:
# Importation des modules dont on aura besoin
import os
import time
import re
import numpy as np
import pandas as pd
import matplotlib as pl
import matplotlib.pyplot as plt
import seaborn as sns
import pprint

# Change le format d'affichage des floats pour être plus lisible
pd.set_option('display.float_format', lambda x: '%.3f' % x)

# Force l'affichage de toutes les colonnes et de toutes les lignes dans les fonctions head/tail/dtypes/etc... 
pd.set_option('display.max_columns',  None)     # Afficher toutes les colonnes
pd.set_option('display.max_rows',     None)     # Afficher toutes les lignes
pd.set_option('display.max_colwidth', 40)       # Limite sur la largeur des colonnes

# Défini les tailles par défaut des polices de caractères pour tous les diagrammes matplotlib
plt.rcParams.update({'font.size': 10}) 
plt.rcParams.update({'axes.titlesize': 12})
plt.rcParams.update({'axes.labelsize': 10})
plt.rcParams.update({'xtick.labelsize': 8})
plt.rcParams.update({'ytick.labelsize': 8})
plt.rcParams.update({'legend.fontsize': 8})

# Défini les grilles par défaut pour tous les diagrammes matplotlib
plt.rcParams.update({"axes.grid" : True, 'grid.color': 'green', 'grid.linestyle':'--', 'grid.linewidth':0.5})

In [13]:
WANTED_PATH="D:\\Dominique\\GIT\\OC_DS_P7"
os.chdir(WANTED_PATH)
print("Répertoire courant : "+os.getcwd())

API_FOLDER = "levray_dominique_3_dossier_code_072024"
ZIP_TEST_DATA_FILENAME = "test_data.zip"                 # fichier contenant les données de test

data_df  = pd.read_csv(API_FOLDER+"/"+ZIP_TEST_DATA_FILENAME,  sep=',', encoding='utf-8',compression='zip')
print(f"\tTaille de data_df={data_df.shape}")


Répertoire courant : D:\Dominique\GIT\OC_DS_P7
	Taille de data_df=(61503, 314)


In [23]:
data_df[data_df['TARGET']==0].head(10)

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH_x,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,EXT_SOURCE_1_x,EXT_SOURCE_2_x,EXT_SOURCE_3_x,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,TOTALAREA_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,_SEC_DAYS_CREDIT,_SEC_DAYS_CREDIT_ENDDATE,_SEC_DAYS_CREDIT_UPDATE,_SEC_CREDIT_DAY_OVERDUE,_SEC_DAYS_ENDDATE_FACT,_SEC_AMT_CREDIT_SUM,_SEC_AMT_CREDIT_SUM_DEBT,_SEC_AMT_CREDIT_SUM_OVERDUE,_SEC_AMT_CREDIT_SUM_LIMIT,_SEC_CNT_CREDIT_PROLONG,_SEC_CREDITS_CLOS,_SEC_INCIDENTS_CREDITS_CLOS,_SEC_AMT_BALANCE,_SEC_AMT_CREDIT_LIMIT_ACTUAL,_SEC_AMT_DRAWINGS_ATM_CURRENT,_SEC_AMT_DRAWINGS_CURRENT,_SEC_AMT_DRAWINGS_OTHER_CURRENT,_SEC_AMT_DRAWINGS_POS_CURRENT,_SEC_AMT_INST_MIN_REGULARITY,_SEC_AMT_PAYMENT_CURRENT,_SEC_AMT_PAYMENT_TOTAL_CURRENT,_SEC_AMT_RECEIVABLE_PRINCIPAL,_SEC_AMT_RECIVABLE,_SEC_AMT_TOTAL_RECEIVABLE,_SEC_CNT_DRAWINGS_ATM_CURRENT,_SEC_CNT_DRAWINGS_CURRENT,_SEC_CNT_DRAWINGS_OTHER_CURRENT,_SEC_CNT_DRAWINGS_POS_CURRENT,_SEC_CNT_INSTALMENT_MATURE_CUM,_SEC_CNT_INSTALMENT,_SEC_CNT_INSTALMENT_FUTURE,_SEC_CREDITS_ACTIFS,_SEC_INCIDENTS_CREDITS_ACTIFS,_SEC_MONTHS_BALANCE,DAYS_EMPLOYED_ANOM,NAME_TYPE_SUITE_Children,NAME_TYPE_SUITE_Family,NAME_TYPE_SUITE_Groupofpeople,NAME_TYPE_SUITE_Other_A,NAME_TYPE_SUITE_Other_B,NAME_TYPE_SUITE_Spousepartner,NAME_TYPE_SUITE_Unaccompanied,NAME_INCOME_TYPE_Businessman,NAME_INCOME_TYPE_Commercialassociate,NAME_INCOME_TYPE_Maternityleave,NAME_INCOME_TYPE_Pensioner,NAME_INCOME_TYPE_Stateservant,NAME_INCOME_TYPE_Student,NAME_INCOME_TYPE_Unemployed,NAME_INCOME_TYPE_Working,NAME_EDUCATION_TYPE_Academicdegree,NAME_EDUCATION_TYPE_Highereducation,NAME_EDUCATION_TYPE_Incompletehigher,NAME_EDUCATION_TYPE_Lowersecondary,NAME_EDUCATION_TYPE_Secondarysecondaryspecial,NAME_FAMILY_STATUS_Civilmarriage,NAME_FAMILY_STATUS_Married,NAME_FAMILY_STATUS_Separated,NAME_FAMILY_STATUS_Singlenotmarried,NAME_FAMILY_STATUS_Unknown,NAME_FAMILY_STATUS_Widow,NAME_HOUSING_TYPE_Coopapartment,NAME_HOUSING_TYPE_Houseapartment,NAME_HOUSING_TYPE_Municipalapartment,NAME_HOUSING_TYPE_Officeapartment,NAME_HOUSING_TYPE_Rentedapartment,NAME_HOUSING_TYPE_Withparents,OCCUPATION_TYPE_Accountants,OCCUPATION_TYPE_Cleaningstaff,OCCUPATION_TYPE_Cookingstaff,OCCUPATION_TYPE_Corestaff,OCCUPATION_T